In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **Selection of the top 20 players**

We will not work with the entire dataset but with the shots coming from the top 20 players that we will select as follows

In [ ]:
import pandas as pd

# Load statistics of each NBA player from 1950
df_season = pd.read_csv('/content/gdrive/My Drive/Projet NBA/Datasets/Seasons_Stats.csv')
df_season.tail()

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24686,24686,2017.0,Cody Zeller,PF,24.0,CHO,62.0,58.0,1725.0,16.7,...,0.679,135.0,270.0,405.0,99.0,62.0,58.0,65.0,189.0,639.0
24687,24687,2017.0,Tyler Zeller,C,27.0,BOS,51.0,5.0,525.0,13.0,...,0.564,43.0,81.0,124.0,42.0,7.0,21.0,20.0,61.0,178.0
24688,24688,2017.0,Stephen Zimmerman,C,20.0,ORL,19.0,0.0,108.0,7.3,...,0.600,11.0,24.0,35.0,4.0,2.0,5.0,3.0,17.0,23.0
24689,24689,2017.0,Paul Zipser,SF,22.0,CHI,44.0,18.0,843.0,6.9,...,0.775,15.0,110.0,125.0,36.0,15.0,16.0,40.0,78.0,240.0
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,...,0.653,41.0,118.0,159.0,30.0,14.0,33.0,30.0,66.0,284.0


In [ ]:
# Filter for seasons played from 2008 onwards
df = df_season[df_season['Year'] >= 2008]

# Filter for players still active in 2017 to get the most recent data
df_players = pd.read_csv('/content/gdrive/My Drive/Projet NBA/Datasets/player_data.csv')
df_players = df_players[df_players['year_end']==2018]
df_season = df_season[df_season['Player'].isin(df_players['name'])]

# Calculate the average games played per season for each player
avg_games = df_season.groupby('Player')['G'].mean()

# Filter for players who had an average of more than 20 games per season
filtered_players = avg_games[avg_games > 20]
df_season = df_season[df_season['Player'].isin(filtered_players.index.tolist())]

In [ ]:
# Check for missing values on the columns of interest
df_season[['G', 'AST', 'TRB', 'PER', 'PTS']].isnull().any()

G      False
AST    False
TRB    False
PER    False
PTS    False
dtype: bool

In [ ]:
# Obtain the 20 top players by calculating a score consireing points, assists, rebounds and PER
career_stats = df_season.groupby("Player").agg({
    "G": sum,
    "PTS": sum,
    "TRB": sum,
    "AST": sum,
    "PER": "mean"
    })

# add a column for points, rebounds and assists per game
career_stats["PPG"] = career_stats.PTS / career_stats.G
career_stats["RPG"] = career_stats.TRB / career_stats.G
career_stats["APG"] = career_stats.AST / career_stats.G

# Group the data by player and calculate the weighted average of PPG, APG, RPG and PER
weights = {'PPG': 0.3, 'APG': 0.1, 'RPG': 0.1, 'PER': 0.5}
df_ranking = career_stats.groupby('Player', as_index=False).apply(lambda x: (x[["PPG", "APG", "RPG", "PER"]] * pd.Series(weights)).sum(axis=1) / sum(weights.values()))
df_ranking = df_ranking.reset_index(name='Score').sort_values(by=['Score'], ascending=False).head(20)
df_ranking

,level_0,Player,Score
209,209,LeBron James,23.419180
190,190,Kevin Durant,21.959196
17,17,Anthony Davis,21.055970
295,295,Russell Westbrook,20.186727
181,181,Karl-Anthony Towns,20.014024
90,90,Dwyane Wade,20.006511
42,42,Chris Paul,19.858663
309,309,Stephen Curry,19.608537
68,68,DeMarcus Cousins,19.476019
25,25,Blake Griffin,19.169305


# **Shots dataset filtered for 20 top players**

In [ ]:
# Loading dataframe "NBA shot Locations"
df_shots = pd.read_csv('/content/gdrive/My Drive/Projet NBA/Datasets/NBA Shot Locations 1997 - 2020.csv')

# Filter for shots taken from 2012 onwards
df_shots['Game Date'] = pd.to_datetime(df_shots['Game Date'], format='%Y%m%d')
df_shots['year'] = df_shots['Game Date'].dt.year
df_shots = df_shots[df_shots['year'] >= 2008]

# Filtering by the top 20 players of df_ranking
df_shots = df_shots[df_shots['Player Name'].isin(df_ranking['Player'])]

# Confirm that we have our top 20 best players in the shots dataset
print('There are', df_shots['Player Name'].nunique(), 'players')
df_shots['Player Name'].unique()

There are 20 players


array(['Chris Paul', 'Dirk Nowitzki', 'Kevin Durant', 'Pau Gasol',
       'LeBron James', 'Carmelo Anthony', 'Dwyane Wade', 'Dwight Howard',
       'Russell Westbrook', 'Kevin Love', 'Brook Lopez', 'James Harden',
       'Stephen Curry', 'Blake Griffin', 'DeMarcus Cousins',
       'Kyrie Irving', 'Anthony Davis', 'Damian Lillard',
       'Karl-Anthony Towns', 'Joel Embiid'], dtype=object)

# **Categorize and modify some variables**

## **Period**

In [ ]:
# Group overtime periods in a category OT
def group_periods(period):
    if period <= 4:
        return str(period)
    else:
        return 'OT'

df_shots['Period Grouped'] = df_shots['Period'].apply(group_periods)

## **Remaining Minutes and Remaining Seconds**

Will generate a single variable called Total Remaining Seconds

In [ ]:
# Delete rows with 12 remaining minutes
df_shots = df_shots.drop(df_shots[df_shots['Minutes Remaining'] == 12].index)
df_shots['Total Seconds Remaining'] = df_shots['Minutes Remaining'] * 60 + df_shots['Seconds Remaining']

## **Action Type**

Instead of grouping by key words, we will group by accuracy ranges as this helps the model

In [ ]:
import numpy as np
action = df_shots.groupby('Action Type')['Shot Made Flag'].agg(['mean', 'count']).reset_index().rename(columns={'mean' : 'accuracy'})
action['freq'] = action['count']/np.sum(action['count'])
action.drop('count', axis=1, inplace=True)
action.sort_values(by='accuracy', ascending = False, inplace = True)

bins = [0, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
labels = ['action_below_40%', 'action_40-50%', 'action_50-60%', 'action_60-70%', 'action_70-80%', 'action_80-90%', 'action_above_90%']
action['Action Type Grouped'] = pd.cut(action['accuracy'], bins=bins, labels = labels)

# Apply the function to the Shot Type column and create a new column with the grouped categories
df_shots = pd.merge(df_shots, action[['Action Type','Action Type Grouped']], on='Action Type', how = 'left')

# **Data Cleaning**

In [ ]:
# Shots by Shot Zone andd Shot type
grouped = df_shots.groupby(['Shot Zone Basic', 'Shot Type'])
shot_counts = grouped['Game ID'].count()
print(shot_counts)

# Delete incoherent results
df_shots = df_shots[((df_shots['Shot Zone Basic'].str.contains('3')) | (df_shots['Shot Zone Basic'] == 'Backcourt')) & (df_shots['Shot Type'] == '3PT Field Goal') | (~(df_shots['Shot Zone Basic'].str.contains('3')) & (df_shots['Shot Zone Basic'] != 'Backcourt')) & (df_shots['Shot Type'] == '2PT Field Goal')]
grouped = df_shots.groupby(['Shot Zone Basic', 'Shot Type'])
shot_counts = grouped['Game ID'].count()
print(shot_counts)

Shot Zone Basic        Shot Type     
Above the Break 3      2PT Field Goal        5
                       3PT Field Goal    51417
Backcourt              3PT Field Goal      407
In The Paint (Non-RA)  2PT Field Goal    38686
                       3PT Field Goal        1
Left Corner 3          3PT Field Goal     3895
Mid-Range              2PT Field Goal    70943
                       3PT Field Goal       21
Restricted Area        2PT Field Goal    79997
                       3PT Field Goal        3
Right Corner 3         2PT Field Goal        4
                       3PT Field Goal     3183
Name: Game ID, dtype: int64
Shot Zone Basic        Shot Type     
Above the Break 3      3PT Field Goal    51417
Backcourt              3PT Field Goal      407
In The Paint (Non-RA)  2PT Field Goal    38686
Left Corner 3          3PT Field Goal     3895
Mid-Range              2PT Field Goal    70943
Restricted Area        2PT Field Goal    79997
Right Corner 3         3PT Field Goal     3183
Nam

# **Enriching shots dataset with other datasets**

## **Adding 2pt and 3pt precision for each player**

In [ ]:
player_stats = df_season.groupby("Player").agg({
    "2P%": "mean",
    "3P%": "mean"
    })

player_stats = player_stats[player_stats.index.isin(df_ranking['Player'])]

df = df_shots.merge(player_stats, left_on = 'Player Name', right_on = player_stats.index)

In [ ]:
# We can also add a similar variable from the same Shots dataset
grouped = df.groupby(['Player ID', 'Shot Type'])['Shot Made Flag'].mean()

# Use unstack() to create separate columns for 2pt and 3pt accuracy
result = grouped.unstack(level=1)
result.columns = [f"{col}_accuracy" for col in result.columns]

# Merge the result back to the original dataframe
df = pd.merge(df, result, on='Player ID', how='left')

## **Adding height, weight, position and age of the player at the time of the shot**

In [ ]:
# creating a column with a continous value of height
df_players[['height-ft', 'height-inch']] = df_players['height'].str.split('-', expand=True).astype('int')
df_players['height-inch'] = df_players['height-inch'] * 0.0833
df_players['height'] = round(df_players['height-ft'] + df_players['height-inch'], 2)
df_players = df_players.drop(['height-ft', 'height-inch'], axis=1)

# creating a column birth year
df_players['birth_year'] = pd.to_datetime(df_players['birth_date']).dt.year

# merging datasets
df = df.merge(df_players[['name', 'position', 'height', 'weight', 'birth_year']], left_on='Player Name', right_on='name')
df = df.drop(['name'], axis=1)

# creating a column age
df['age'] = df['year'] - df['birth_year']

## **Adding the performance of the team at the time of the shot**

In [ ]:
df_team_ranking = pd.read_csv('/content/gdrive/My Drive/Projet NBA/Datasets/ranking.csv')
df_team_ranking['STANDINGSDATE'] = pd.to_datetime(df_team_ranking["STANDINGSDATE"], format = '%Y-%m-%d')
df_team_ranking= df_team_ranking[['TEAM_ID', 'STANDINGSDATE', 'G', 'W', 'L', 'W_PCT']]

df = df.merge(df_team_ranking, left_on=['Game Date', 'Team ID'], right_on = ['STANDINGSDATE', 'TEAM_ID']).drop(['STANDINGSDATE', 'TEAM_ID'], axis=1)

## **Adding if the shooting team is home or away**

In [ ]:
df_teams = pd.read_csv('/content/gdrive/My Drive/Projet NBA/Datasets/teams.csv')
df = pd.merge(df, df_teams[['TEAM_ID', 'ABBREVIATION']], left_on='Team ID', right_on='TEAM_ID', how='left')
df['Shot_Team'] = df.apply(lambda row: 'Home' if row['ABBREVIATION'] == row['Home Team'] else 'Away', axis=1)
df = df.drop('TEAM_ID', axis=1)

## **Adding a variable to indicate if the player is 'On-fire'**


In [ ]:
# N et X sont définis dans les 2 variables qu'on pourra modifier : 
nbgames_activ_on_fire = 3 # Cette variable servira à choisir sur combien de matchs consécutifs on choisit d'activer l'option "on_fire". Ici elle s'active sur 3 matchs consécutifs.
pct_activ_on_fire = 0.05 # On choisit à partir de quel pourcentage d'augmentation l'option on_fire s'active. 
seuil_nb_shots = 5 # On choisit à partir de combien de shots dans un match la variable on_fire peut s'activer.

# On trie les matchs et les actions par ordre croissant pour avoir les matchs par ordre temporel : 
df = df.sort_values(by = ['Game ID'], ascending = True)

# Calcul de la moyenne de réussite de shot globale pour chaque joueur et ajout de la nouvelle colonne dans df :
player_accuracy = df.groupby('Player ID')['Shot Made Flag'].agg('mean')
df['player_accuracy'] = df['Player ID'].map(player_accuracy)

# Calcul de la moyenne de réussite sur chaque match pour chaque joueur :
game_player_accuracy = df.groupby(['Player ID', 'Game ID'])['Shot Made Flag'].agg('mean').reset_index().rename(columns={'Shot Made Flag' : 'game_player_accuracy'}) 

# Ajout de la nouvelle colonne 'game_player_accuracy' dans df avec les valeurs correspondantes au Player ID et au Game ID :
df['game_player_accuracy'] = df.apply(lambda x: game_player_accuracy.loc[(game_player_accuracy['Player ID'] == x['Player ID']) 
                                                                         & (game_player_accuracy['Game ID'] == x['Game ID']), 'game_player_accuracy'].values[0], axis=1)

# Calcul du nombre de tir pris par joueur pour chaque match : 
nb_shot_per_game = df.groupby(['Player ID', 'Game ID'])['Player Name'].count().reset_index().rename(columns={'Player Name' : 'nb_shot_per_game'})

# Ajout de la nouvelle colonne 'nb_shot_per_game' dans df :
df['nb_shot_per_game'] = df.apply(lambda x: nb_shot_per_game.loc[(nb_shot_per_game['Player ID'] == x['Player ID']) 
                                                                         & (nb_shot_per_game['Game ID'] == x['Game ID']), 'nb_shot_per_game'].values[0], axis=1)

# Création de la colonne 'game_on_fire' qui affichera 1 si l'accuracy sur le match a une augmentation significative et si le nombre de tir pris dans le match est supérieur au seuil définit, 0 sinon : 
df['game_on_fire'] = df.apply(lambda row: 1 if row['game_player_accuracy'] != 0 # précaution pour éviter la division par 0
                              and (row['game_player_accuracy'] - row['player_accuracy']) / row['game_player_accuracy'] > pct_activ_on_fire # condition de +X% d'augmentation d'accuracy
                              and row['nb_shot_per_game'] >= seuil_nb_shots # condition du seuil de nombre de tir pris
                              else 0, axis=1)

# On regroupe ces données par joueurs et par matchs pour ensuite vérifier si N matchs consécutifs sont en 'games_on_fire'==1 :
games_on_fire = df.groupby(['Player ID', 'Game ID'])['game_on_fire'].agg('max').to_frame().reset_index()

# Pour cela on créé une nouvelle colonne qui fera la somme des matchs "on_fire" sur une fenêtre de taille N définie avec la variable 'nbgames_activ_on_fire'.
games_on_fire['rolling_sum'] = games_on_fire.groupby('Player ID')['game_on_fire'].rolling(window=nbgames_activ_on_fire).sum().reset_index(0, drop=True)

# Création d'une fonction qui remplacera une cellule par 1 si N matchs consécutifs sont en 'games_on_fire'==1 :
def replace_onfire(rolling_sum):
    if rolling_sum >= nbgames_activ_on_fire:
        return 1
    else:
        return 0

# appliquer la condition à la colonne 'rolling_sum' et appliquer les valeurs à la colonne 'on_fire' :
games_on_fire['on_fire'] = games_on_fire['rolling_sum'].apply(replace_onfire)

# Ajout de la nouvelle colonne 'on_fire' dans df avec les valeurs correspondantes au Player ID et au Game ID :
df['on_fire'] = df.apply(lambda x: games_on_fire.loc[(games_on_fire['Player ID'] == x['Player ID']) & (games_on_fire['Game ID'] == x['Game ID']), 'on_fire'].values[0], axis=1)

# **Saving the final dataframe**

In [ ]:
df.to_csv('/content/gdrive/My Drive/Projet NBA/Datasets/df.csv')